# Synodal Bible & Mother

## Summary

In [1]:
# Code summary of below

In [2]:
!python --version

Python 3.11.4


## Import

### Libraries

In [3]:
from backend import *
import multiprocessing as mp
from multiprocessing import Pool, Process
import pickle
num_cores = int(mp.cpu_count()-2)
print(f"Number of cores : {num_cores}")

Number of cores : 38


In [4]:
#%matplotlib inline

### Definitions

In [5]:
texts = '../texts/fiction/utf8/'

libCols = ['author','pub_year','title','text']
tokenOHCO = ['w_id','part_num','para_num', 'sent_num', 'token_num']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', \
             'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

In [6]:
motherTokenDf = pd.read_pickle('./proc/MotherTokendf.pkl')
bibleTokenDf = pd.read_pickle('./proc/BibleTokenDf.pkl')
confessionTokenDf = pd.read_pickle('./proc/ConfessionTokenDf.pkl')

In [7]:
good_pos = ['NOUN', 'ADJ', 'VERB', 'PROPN']

In [8]:
# add stopword boolean, True if all attribute columns are null
#motherTokenDf['stopword'] = ~motherTokenDf.loc[:, tokenCols[9:]].any(axis=1)
motherTokenDf['stopword'] = ~motherTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (motherTokenDf.shape[0]+1)), name='id')
motherTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

bibleTokenDf['stopword'] = ~bibleTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (bibleTokenDf.shape[0]+1)), name='id')
bibleTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

confessionTokenDf['stopword'] = ~confessionTokenDf.pos.isin(good_pos)
TokenDfIdx = pd.Index(range(1, (confessionTokenDf.shape[0]+1)), name='id')
confessionTokenDf.index = TokenDfIdx# = TokenDf.set_index(['p_id', 'token_id'])

## Bible and Mother Token Dataframes

In [9]:
bibleTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,1,В,1_1,1_2,case,ADP,в,None,...,None,None,None,None,None,None,None,None,None,True
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675064,37086,32,38,Христа,1_5,1_4,flat:name,PROPN,христос,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
675065,37086,39,41,со,1_6,1_8,case,ADP,с,None,...,None,None,None,None,None,None,None,None,None,True
675066,37086,42,47,всеми,1_7,1_8,det,DET,весь,None,...,Ins,None,None,None,Plur,None,None,None,None,True


In [10]:
bibleTokenDf.loc[bibleTokenDf.stopword == False]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2,1,2,8,начале,1_2,1_3,obl,NOUN,начало,Inan,...,Loc,None,Neut,None,Sing,None,None,None,None,False
3,1,9,17,сотворил,1_3,1_0,root,VERB,сотворить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
4,1,18,21,Бог,1_4,1_3,nsubj,PROPN,бог,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
5,1,22,26,небо,1_5,1_3,obj,NOUN,небо,Inan,...,Acc,None,Neut,None,Sing,None,None,None,None,False
7,1,29,34,землю,1_7,1_5,conj,NOUN,земля,Inan,...,Acc,None,Fem,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675060,37086,0,9,Благодать,1_1,1_2,nsubj,NOUN,благодать,Inan,...,Nom,None,Fem,None,Sing,None,None,None,None,False
675061,37086,10,17,Господа,1_2,1_1,appos,PROPN,господь,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False
675063,37086,25,31,Иисуса,1_4,1_1,nmod,PROPN,иисус,Anim,...,Gen,None,Masc,None,Sing,None,None,None,None,False


In [11]:
motherTokenDf#.set_index(['p_id','token_id'])

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,7,13,Каждый,1_1,1_2,det,DET,каждый,None,...,Acc,None,Masc,None,Sing,None,None,None,None,True
2,1,14,18,день,1_2,1_28,obl,NOUN,день,Inan,...,Acc,None,Masc,None,Sing,None,None,None,None,False
3,1,19,22,над,1_3,1_5,case,ADP,над,None,...,None,None,None,None,None,None,None,None,None,True
4,1,23,30,рабочей,1_4,1_5,amod,ADJ,рабочий,None,...,Ins,Pos,Fem,None,Sing,None,None,None,None,False
5,1,31,40,слободкой,1_5,1_28,obl,NOUN,слободка,Inan,...,Ins,None,Fem,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86005,4436,7,13,Кто-то,1_1,1_2,nsubj,PRON,кто-то,None,...,Nom,None,None,None,None,None,None,None,None,True
86006,4436,14,21,ответил,1_2,1_0,root,VERB,ответить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False
86007,4436,22,24,ей,1_3,1_2,iobj,PRON,она,None,...,Dat,None,Fem,None,Sing,3,None,None,None,True


In [12]:
motherTokenDf.loc[motherTokenDf.stopword == False]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2,1,14,18,день,1_2,1_28,obl,NOUN,день,Inan,...,Acc,None,Masc,None,Sing,None,None,None,None,False
4,1,23,30,рабочей,1_4,1_5,amod,ADJ,рабочий,None,...,Ins,Pos,Fem,None,Sing,None,None,None,None,False
5,1,31,40,слободкой,1_5,1_28,obl,NOUN,слободка,Inan,...,Ins,None,Fem,None,Sing,None,None,None,None,False
7,1,44,50,дымном,1_8,1_34,conj,NOUN,дымный,Inan,...,Loc,None,Masc,None,Sing,None,None,None,None,False
8,1,52,60,масляном,1_10,1_11,amod,VERB,масляный,None,...,Loc,None,Masc,None,Sing,None,Pres,Part,Act,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86003,4434,54,61,хрипела,2_2,2_0,root,ADJ,хрипеть,None,...,None,Pos,Fem,None,Sing,None,None,None,None,False
86004,4435,10,20,Несчастные,1_2,1_0,root,ADJ,несчастный,None,...,Nom,Pos,None,None,Plur,None,None,None,None,False
86006,4436,14,21,ответил,1_2,1_0,root,VERB,ответить,None,...,None,None,Masc,Ind,Sing,None,Past,Fin,Act,False


In [13]:
confessionTokenDf

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
1,1,10,19,Позвольте,1_2,1_0,root,VERB,позволить,None,...,None,None,None,Imp,Plur,2,None,Fin,Act,False
2,1,20,30,рассказать,1_3,1_2,xcomp,VERB,рассказать,None,...,None,None,None,None,None,None,None,Inf,Act,False
3,1,31,36,жизнь,1_4,1_3,obj,NOUN,жизнь,Inan,...,Acc,None,Fem,None,Sing,None,None,None,None,False
4,1,37,40,мою,1_5,1_17,det,DET,мой,None,...,Acc,None,Fem,None,Sing,None,None,None,None,True
5,1,42,49,времени,1_7,1_10,obj,NOUN,время,Inan,...,Gen,None,Neut,None,Sing,None,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47480,2360,283,291,великого,1_49,1_50,amod,ADJ,великий,None,...,Gen,Pos,Neut,None,Sing,None,None,None,None,False
47481,2360,292,296,дела,1_50,1_11,obl,NOUN,дело,Inan,...,Gen,None,Neut,None,Sing,None,None,None,None,False
47482,2360,300,310,всемирного,1_52,1_53,amod,ADJ,всемирный,None,...,Gen,Pos,Masc,None,Sing,None,None,None,None,False


In [14]:
lemmaDf = bibleTokenDf[['lemma']].loc[bibleTokenDf.stopword == False].join(motherTokenDf[['lemma']].loc[motherTokenDf.stopword == False], on='id', lsuffix='_bible', rsuffix='_mother')
lemmaDf

,lemma_bible,lemma_mother
id,,
2,начало,день
3,сотворить,NaN
4,бог,рабочий
5,небо,слободка
7,земля,дымный
...,...,...
675060,благодать,NaN
675061,господь,NaN
675063,иисус,NaN


## Shared Lemmas

In [15]:
bible_lemmas = bibleTokenDf.loc[bibleTokenDf.stopword == False].lemma
mother_lemmas = motherTokenDf.loc[motherTokenDf.stopword == False].lemma
confession_lemmas = confessionTokenDf.loc[confessionTokenDf.stopword == False].lemma

In [16]:
common_lemmas = []
common_lemmas_count = 0

for lemma in set(mother_lemmas):
    #print(lemma)
    if lemma in bible_lemmas.unique():
        common_lemmas.append(lemma)
        common_lemmas_count += 1

print(f"Shared unique words between Mother and the Bible: "+str(common_lemmas_count)+
      ", that's ~"+str(round(((common_lemmas_count/len(set(mother_lemmas)))*100),2))+"%")

Shared unique words between Mother and the Bible: 3534, that's ~48.68%


In [17]:
common_lemmas = []
common_lemmas_count = 0

for lemma in set(confession_lemmas):
    #print(lemma)
    if lemma in bible_lemmas.unique():
        common_lemmas.append(lemma)
        common_lemmas_count += 1

print(f"Shared unique words between Confession and the Bible: "+str(common_lemmas_count)+
      ", that's ~"+str(round(((common_lemmas_count/len(set(confession_lemmas)))*100),2))+"%")

Shared unique words between Confession and the Bible: 3227, that's ~53.88%


In [18]:
len(set(confession_lemmas))/47484

0.1261266953078932

In [19]:
bible_vc = bible_lemmas.to_frame().value_counts().to_frame()
mother_vc = mother_lemmas.to_frame().value_counts().to_frame()
confession_vc = confession_lemmas.to_frame().value_counts().to_frame()

In [51]:
bible_vc.reset_index().loc[bible_vc.reset_index().lemma == 'мать']

,lemma,count
137,мать,366


In [54]:
mother_vc.reset_index().loc[mother_vc.reset_index().lemma == 'отец']

,lemma,count
229,отец,34


In [57]:
confession_vc.reset_index().loc[confession_vc.reset_index().lemma == 'город']

,lemma,count
184,город,21


In [20]:
countDf = mother_vc.join(bible_vc, lsuffix='_mother', rsuffix='_bible')
countDf = countDf.join(confession_vc, lsuffix='_mother', rsuffix='_confession')
countDf = countDf.rename({'count':'count_confession'}, axis=1)

In [22]:
motherLemmaDf = countDf[:25].fillna(0).astype(int, errors='ignore')[['count_mother', 'count_bible']].style.background_gradient(cmap='plasma')
confessionLemmaDf = countDf.sort_values('count_confession', ascending=False)[:25].fillna(0).astype(int, errors='ignore')[['count_confession', 'count_bible']].style.background_gradient(cmap='plasma')
bibleLemmaDf = countDf.sort_values('count_bible', ascending=False)[:25].fillna(0).astype(int, errors='ignore').style.background_gradient(cmap='plasma')

In [23]:
display_side_by_side(motherLemmaDf, bibleLemmaDf, confessionLemmaDf)

,count_mother,count_bible
lemma,,
мать,983,366
сказать,697,5844
человек,680,2349
говорить,569,3732
рука,478,1889
павел,414,155
глаз,412,549
лицо,396,683
голова,371,429


In [24]:
def compare_list_frequency(*args,
                           consider_order=False,
                           consider_duplicates=True,
                           handle_unhashable=False,
                           key=None,
                           frequency_format="dict"):    
    list_hit_list = []
    for arg in args:
        list_hit_dict = {}
        other_args = args[:args.index(arg)] + args[args.index(arg)+1:]
        for word in set(arg):
            word_hit_count = 0
            for other_arg in other_args:
                word_hit_count += list(other_arg).count(word)
            if word_hit_count > 0:
                list_hit_dict[word] = word_hit_count
            #print(f"{word}: {list(bible_lemmas[:1000]).count(word)}")
        list_hit_list.append(list_hit_dict)

    return(list_hit_list)

In [25]:
comparison_lists = compare_list_frequency(list(mother_lemmas), list(bible_lemmas), list(confession_lemmas))

In [26]:
mother_lemmas.value_counts().to_frame()

,count
lemma,
мать,983
сказать,697
человек,680
говорить,569
рука,478
...,...
нагнать,1
отравить,1
оголить,1


In [27]:
compDf = pd.DataFrame(comparison_lists).transpose(copy=True).fillna(0).astype(int)
compDf.columns = ['mother', 'bible', 'confession']
compDf = compDf.fillna(0).astype(int, errors='ignore')
compDf.sample(25).style.background_gradient(cmap='plasma')

,mother,bible,confession
кверху,2,4,4
соленый,14,2,0
копье,0,1,84
молитва,197,25,182
значить,73,48,81
яко,7,0,1
захотеть,77,7,82
молиться,203,30,197
купить,82,15,85
поглаживать,2,0,3


In [28]:
#mcompDf = compDf.sort_values('mother', axis=0, ascending=False)[:25].style.background_gradient(cmap='plasma')
bcompDf = compDf.sort_values('bible', axis=0, ascending=False)[:25].style.background_gradient(cmap='plasma')
#ccompDf = compDf.sort_values('confession', axis=0, ascending=False)[:25].style.background_gradient(cmap='plasma')

In [29]:
bcompDf

,mother,bible,confession
человек,2763,1094,3029
говорить,4220,1057,4301
мать,400,1017,1349
сказать,5969,822,6541
рука,2006,595,2367
глаз,687,550,961
лицо,816,529,1079
видеть,1186,465,1185
голова,513,455,800
павел,155,414,0


In [30]:
def ListSplit(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window(lemma_array, window_size):
    token_window_array = np.array([])
    token_window_array = np.array([np.array(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def get_sim_count(child_window_array, parent_window_array, order_matters=False):
    work_sim_count = 0
    work_sim = []
    for child_window in child_window_array:
        child_sim = {}
        #print(f"child window: {child_window}")
        child_window_sim_total_count = 0
        for parent_window in parent_window_array:
            #print(f"parent window: {parent_window}")
            child_window_sim_sub_count = 0
            for i in range(len(child_window)):
                if order_matters:
                    if child_window[1][i] == parent_window[1][i]:
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                else:
                    if child_window[1][i] in parent_window[1]:
                        #print(f"{child_window[i]} is in parent!")
                        child_window_sim_total_count+=1
                        child_window_sim_sub_count+=1
                    #else:
                        #print(f"{child_window[i]} is not in parent!")
            child_sim.update({parent_window[0]: child_window_sim_sub_count})
        #print(f"{child_window}: {child_window_sim_total_count}")
        work_sim.append({child_window_sim_total_count: child_sim})
    return work_sim

def dna_test_windows(child_lemmas, parent_lemmas, window_size=10):
    sim_counter = 0
    #sim_list = np.array([])
    parent_window_array = token_window(parent_lemmas, window_size)
    child_window_array = token_window(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    #print(sim_list)
    return sim_list

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=10):
    #child_lemmas, parent_lemmas = cnp_lemmas[0], cnp_lemmas[1]
    #print(f"Child lemmas: {child_lemmas}")
    #print(f"Parent lemmas: {parent_lemmas[:10]}")
    sim_counter = 0
    sim_list = np.array([])
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = get_sim_count(child_window_array, parent_window_array)
    return sim_list

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

In [31]:
def list_split(lst, numGroups, sort=True): 
    """Takes a list and a number and splits the 
    list into evenly divided n groups (as much as possible)"""
    # choose to sort list by ascending
    if sort: 
        lst.sort()
    # get length of list given to sort
    listLen = len(lst)
    # groupLen is the maximum number of items per group to allow for the most groups <= numGroups in a list (numGroups-1 OR numGroups)
    groupLen = (listLen//numGroups) + (listLen % numGroups > 0)
    # yield generator object of nested listed with length of numGroups 
    for i in range(0,len(lst), groupLen): 
        yield lst[i:i+groupLen]

def token_window_mp(lemma_array, window_size):
    #token_window_array = np.array([])
    token_window_array = enumerate([list(lemma_array[i:i+window_size]) for i in range(len(lemma_array)-(window_size-1))])
    return token_window_array

def window_count_sim_mp(child_window_array, parent_window_array, order_matters=False):
    sim_count_tuples = {}
    for (c_sub_array_id, c_sub_array_list) in child_window_array:
        c_sub_sim_count = 0
        for (p_sub_array_id, p_sub_array_list) in parent_window_array:
            c_p_sim_count = 0
            for i in range(len(c_sub_array_list)):
                if order_matters:
                    if c_sub_array_list[i] == p_sub_array_list[i]:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
                else:
                    if c_sub_array_list[i] in p_sub_array_list:
                        c_sub_sim_count += 1
                        c_p_sim_count += 1
            if order_matters: 
                if c_p_sim_count > 1: 
                    sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
            elif c_p_sim_count > 2: 
                sim_count_tuples[(c_sub_array_id, p_sub_array_id)] = c_p_sim_count
    return sim_count_tuples

def dna_test_windows_mp(child_lemmas, parent_lemmas, window_size=10):
    sim_list = {}
    parent_window_array = token_window_mp(parent_lemmas, window_size)
    child_window_array = token_window_mp(child_lemmas, window_size)
    sim_list = window_count_sim_mp(child_window_array, parent_window_array)
    return sim_list

In [32]:
child_window_array = list(token_window_mp(mother_lemmas, 7))
parent_window_array = list(token_window_mp(bible_lemmas, 7))

In [33]:
len(child_window_array)

46830

In [34]:
len(parent_window_array)

345084

In [35]:
%%time
if __name__ == "__main__":
    pool = Pool(processes=num_cores)
    #workerAssignments = ((x, bible_lemmas) for x in ListSplit(mother_lemmas[:198], num_cores, False))
    starmap_list = list(zip(list_split(child_window_array, num_cores), list_split(parent_window_array, num_cores)))
    print(f"Splitting into {num_cores} groups")
    start_time = time.time()
    sim_tuples = pool.starmap(window_count_sim_mp, starmap_list, chunksize=2)
    print(f"Finished in {round(time.time()-start_time, 3)} seconds.")
    pool.close()
    pool.join()
    #print(sim_tuples)

Splitting into 38 groups
Finished in 25.223 seconds.
CPU times: user 317 ms, sys: 797 ms, total: 1.11 s
Wall time: 26.2 s


In [36]:
for i in range(len(sim_tuples)):
    print(f"len({i}): {len(sim_tuples[i])}")

len(0): 721
len(1): 2684
len(2): 167
len(3): 1183
len(4): 621
len(5): 3337
len(6): 655
len(7): 1221
len(8): 3692
len(9): 2820
len(10): 847
len(11): 2071
len(12): 2428
len(13): 1450
len(14): 977
len(15): 813
len(16): 1999
len(17): 604
len(18): 341
len(19): 941
len(20): 461
len(21): 817
len(22): 1468
len(23): 511
len(24): 990
len(25): 1136
len(26): 118
len(27): 2070
len(28): 220
len(29): 331
len(30): 603
len(31): 3293
len(32): 571
len(33): 739
len(34): 1147
len(35): 487
len(36): 461
len(37): 602


In [37]:
total_sim_tuples = {}
for t in sim_tuples:
    total_sim_tuples.update(t)

In [38]:
len(total_sim_tuples)

45597

In [39]:
sorted_sim_tuples = sorted(total_sim_tuples.items(), key=lambda x:x[1], reverse=True)
sorted_sim_tuples

[((5232, 40313), 6),
 ((5232, 40314), 6),
 ((5232, 40315), 6),
 ((5232, 40316), 6),
 ((5232, 40317), 6),
 ((5232, 40318), 6),
 ((5232, 42909), 6),
 ((5232, 42910), 6),
 ((5232, 42911), 6),
 ((5232, 42912), 6),
 ((15376, 114312), 6),
 ((15376, 114313), 6),
 ((15376, 115154), 6),
 ((15376, 115157), 6),
 ((15376, 115158), 6),
 ((15376, 115159), 6),
 ((15376, 115160), 6),
 ((5231, 40313), 5),
 ((5231, 40314), 5),
 ((5231, 40315), 5),
 ((5231, 40316), 5),
 ((5231, 40317), 5),
 ((5231, 40318), 5),
 ((5231, 42909), 5),
 ((5231, 42910), 5),
 ((5231, 42911), 5),
 ((5231, 42912), 5),
 ((5233, 40313), 5),
 ((5233, 40314), 5),
 ((5233, 40315), 5),
 ((5233, 40316), 5),
 ((5233, 40317), 5),
 ((5233, 40318), 5),
 ((5233, 42909), 5),
 ((5233, 42910), 5),
 ((5233, 42911), 5),
 ((5233, 42912), 5),
 ((5234, 40313), 5),
 ((5234, 40314), 5),
 ((5234, 40315), 5),
 ((5234, 40316), 5),
 ((5234, 40317), 5),
 ((5234, 40318), 5),
 ((5234, 42909), 5),
 ((5234, 42910), 5),
 ((5234, 42911), 5),
 ((5234, 42912), 5),

In [40]:
motherTokenDf.loc[motherTokenDf.p_id == 693]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
14305,693,10,12,Мы,1_2,1_7,nsubj,PRON,мы,None,...,Nom,None,None,None,Plur,1,None,None,None,True
14306,693,13,19,всегда,1_3,1_7,advmod,ADV,всегда,None,...,None,Pos,None,None,None,None,None,None,None,True
14307,693,20,21,и,1_4,1_13,cc,CCONJ,и,None,...,None,None,None,None,None,None,None,None,None,True
14308,693,22,27,везде,1_5,1_7,advmod,ADV,везде,None,...,None,Pos,None,None,None,None,None,None,None,True
14309,693,31,37,первые,1_7,1_0,root,ADJ,первый,None,...,Nom,Pos,None,None,Plur,None,None,None,None,False
14310,693,38,39,в,1_8,1_9,case,ADP,в,None,...,None,None,None,None,None,None,None,None,None,True
14311,693,40,46,работе,1_9,1_7,nmod,NOUN,работа,Inan,...,Loc,None,Fem,None,Sing,None,None,None,None,False
14312,693,47,48,и,1_10,1_13,cc,CCONJ,и,None,...,None,None,None,None,None,None,None,None,None,True
14313,693,49,51,на,1_11,1_13,case,ADP,на,None,...,None,None,None,None,None,None,None,None,None,True


In [41]:
bibleTokenDf.loc[(bibleTokenDf.p_id == 5374) | (bibleTokenDf.p_id == 5373)]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
103902,5373,0,1,и,1_1,1_2,cc,CCONJ,и,None,...,None,None,None,None,None,None,None,None,None,True
103903,5373,2,7,приди,1_2,1_0,root,VERB,прийти,None,...,None,None,None,Imp,Sing,2,None,Fin,Act,False
103904,5373,8,9,к,1_3,1_4,case,ADP,к,None,...,None,None,None,None,None,None,None,None,None,True
103905,5373,10,21,священникам,1_4,1_2,obl,NOUN,священник,Anim,...,Dat,None,Masc,None,Plur,None,None,None,None,False
103906,5373,22,29,левитам,1_5,1_2,obl,NOUN,левит,Anim,...,Dat,None,Masc,None,Plur,None,None,None,None,False
103907,5373,30,31,и,1_6,1_8,advmod,CCONJ,и,None,...,None,None,None,None,None,None,None,None,None,True
103908,5373,32,33,к,1_7,1_8,case,ADP,к,None,...,None,None,None,None,None,None,None,None,None,True
103909,5373,34,39,судье,1_8,1_14,conj,NOUN,судья,Anim,...,Dat,None,Masc,None,Sing,None,None,None,None,False
103910,5373,41,48,который,1_10,1_26,nsubj,PRON,который,None,...,Nom,None,Masc,None,Sing,None,None,None,None,True


In [42]:
motherTokenDf.loc[motherTokenDf.lemma == "заботиться"]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
2449,76,799,809,заботиться,7_20,7_18,xcomp,VERB,заботиться,None,...,None,None,None,None,None,None,None,Inf,Mid,False
14319,693,85,94,заботится,2_2,2_0,root,VERB,заботиться,None,...,None,None,None,Ind,Sing,3,Pres,Fin,Mid,False


In [43]:
bibleTokenDf.loc[bibleTokenDf.lemma == "поступь"]

,p_id,start,stop,text,token_id,head_id,rel,pos,lemma,anim,...,case,degree,gender,mood,number,person,tense,verb_form,voice,stopword
id,,,,,,,,,,,,,,,,,,,,,
46256,2372,3,10,поступи,1_2,1_0,root,NOUN,поступь,Inan,...,Nom,None,Masc,None,Sing,None,None,None,None,False
103925,5374,2,9,поступи,1_2,1_0,root,NOUN,поступь,Inan,...,Nom,None,Masc,None,Sing,None,None,None,None,False
119604,6062,92,99,поступи,1_20,1_16,conj,NOUN,поступь,Inan,...,Nom,None,Masc,None,Sing,None,None,None,None,False
175707,8552,175,182,поступи,2_18,2_10,conj,NOUN,поступь,Inan,...,Ins,None,Masc,None,Sing,None,None,None,None,False
180260,8780,0,7,поступи,1_1,1_0,root,NOUN,поступь,Inan,...,Gen,None,Masc,None,Sing,None,None,None,None,False
318705,16339,0,7,Поступи,1_1,1_0,root,PROPN,поступь,Anim,...,Nom,None,Masc,None,Sing,None,None,None,None,False
376226,20790,113,121,поступью,1_24,1_26,iobj,NOUN,поступь,Inan,...,Gen,None,Fem,None,Sing,None,None,None,None,False
410746,22473,171,178,поступи,1_37,1_6,nsubj,NOUN,поступь,Inan,...,Nom,None,Masc,None,Plur,None,None,None,None,False
512564,27399,12,19,поступи,1_4,1_16,nsubj,NOUN,поступь,Anim,...,Nom,None,Masc,None,Plur,None,None,None,None,False


In [44]:
child_window_array[7025:7030]

[(7025, ['огонь', 'жить', 'сердце', 'сказать', 'бог', 'слово', 'слово']),
 (7026, ['жить', 'сердце', 'сказать', 'бог', 'слово', 'слово', 'дух']),
 (7027, ['сердце', 'сказать', 'бог', 'слово', 'слово', 'дух', 'разум']),
 (7028, ['сказать', 'бог', 'слово', 'слово', 'дух', 'разум', 'сказать']),
 (7029, ['бог', 'слово', 'слово', 'дух', 'разум', 'сказать', 'павел'])]

In [45]:
parent_window_array[52840:52850]

[(52840,
  ['день', 'спросить', 'сказать', 'рассудить', 'поступь', 'слово', 'сказать']),
 (52841,
  ['спросить',
   'сказать',
   'рассудить',
   'поступь',
   'слово',
   'сказать',
   'место']),
 (52842,
  ['сказать', 'рассудить', 'поступь', 'слово', 'сказать', 'место', 'избрать']),
 (52843,
  ['рассудить', 'поступь', 'слово', 'сказать', 'место', 'избрать', 'господь']),
 (52844,
  ['поступь', 'слово', 'сказать', 'место', 'избрать', 'господь', 'бог']),
 (52845,
  ['слово', 'сказать', 'место', 'избрать', 'господь', 'бог', 'призывать']),
 (52846,
  ['сказать', 'место', 'избрать', 'господь', 'бог', 'призывать', 'имя']),
 (52847, ['место', 'избрать', 'господь', 'бог', 'призывать', 'имя', 'он']),
 (52848,
  ['избрать', 'господь', 'бог', 'призывать', 'имя', 'он', 'постараться']),
 (52849,
  ['господь', 'бог', 'призывать', 'имя', 'он', 'постараться', 'исполнить'])]

In [46]:
pd.DataFrame(child_window_array[7028][1])

,0
0,сказать
1,бог
2,слово
3,слово
4,дух
5,разум
6,сказать


In [47]:
pd.DataFrame.from_dict({x:y for x,y in parent_window_array[:5]})

,0,1,2,3,4
0,начало,сотворить,бог,небо,земля
1,сотворить,бог,небо,земля,земля
2,бог,небо,земля,земля,безвидный
3,небо,земля,земля,безвидный,пустой
4,земля,земля,безвидный,пустой,тьма
5,земля,безвидный,пустой,тьма,бездна
6,безвидный,пустой,тьма,бездна,дух


In [48]:
sorted_consec_sim_tuples = sorted(total_consec_sim_tuples.items(), key=lambda x:x[1], reverse=True)

NameError: name 'total_consec_sim_tuples' is not defined

In [ ]:
sorted_consec_sim_tuples

In [70]:
compareDf = pd.DataFrame()
for x,y in sim_tuples[9:16]:
    #print(child_window_array[x[0]][0])
    #print(child_window_array[x[0]][1])
    compareDf[child_window_array[x[0]][0]] = child_window_array[x[0]][1]
    compareDf[parent_window_array[x[1]][0]] = parent_window_array[x[1]][1]
    #compareDf[x[1]] = parent_window_array[x][1]

compareDf

,5232,42912,15376,114312,114313,115154,115157,115158,115159
0,сказать,сказать,пойти,нафан,пойти,говорить,прозорливец,давидов,сказать
1,павел,господь,сказать,пойти,сказать,господь,давидов,сказать,пойти
2,бояться,моисей,сашенька,сказать,раб,гад,сказать,пойти,сказать
3,бояться,бояться,сказать,раб,мой,прозорливец,пойти,сказать,давид
4,бояться,предать,сказать,мой,давид,давидов,сказать,давид,говорить
5,бояться,народ,говорить,давид,говорить,сказать,давид,говорить,господь
6,сказать,земля,говорить,говорить,господь,пойти,говорить,господь,наказание


In [39]:
with open('./proc/sim_tuples.pkl', 'wb') as file:
    pickle.dump(sorted_sim_tuples, file)

In [ ]:
with open('./proc/consec_sim_tuples.pkl', 'wb') as file:
    pickle.dump(sorted_consec_sim_tuples, file)

In [58]:
with open('./proc/sim_tuples.pkl', 'rb') as file:
    # The protocol version used is detected automatically, so we do not have to specify it.
    sim_tuples = pickle.load(file)

In [31]:
with open('./proc/consec_sim_tuples.pkl', 'rb') as file:
    # The protocol version used is detected automatically, so we do not have to specify it.
    csim_tuples = pickle.load(file)

In [32]:
csim_tuples

[((2284, 17791), 3),
 ((2285, 17792), 3),
 ((2286, 17793), 3),
 ((2348, 13927), 3),
 ((2355, 13399), 3),
 ((2356, 13400), 3),
 ((2357, 13401), 3),
 ((2515, 12268), 3),
 ((2516, 12269), 3),
 ((2517, 12270), 3),
 ((2518, 12271), 3),
 ((4055, 21854), 3),
 ((4056, 21855), 3),
 ((4908, 37762), 3),
 ((5119, 36624), 3),
 ((5120, 36625), 3),
 ((5121, 36626), 3),
 ((5122, 36627), 3),
 ((5123, 36628), 3),
 ((5124, 36629), 3),
 ((5125, 36630), 3),
 ((5126, 36631), 3),
 ((5784, 40092), 3),
 ((5785, 40093), 3),
 ((8163, 53086), 3),
 ((8164, 53087), 3),
 ((8165, 53088), 3),
 ((8166, 53089), 3),
 ((8167, 53090), 3),
 ((8168, 53091), 3),
 ((8169, 53092), 3),
 ((8170, 53093), 3),
 ((8255, 57313), 3),
 ((8256, 57314), 3),
 ((8257, 57315), 3),
 ((8265, 52937), 3),
 ((8265, 55011), 3),
 ((8266, 52938), 3),
 ((8266, 55012), 3),
 ((8267, 55013), 3),
 ((8268, 55014), 3),
 ((8295, 54650), 3),
 ((8296, 54651), 3),
 ((8297, 54652), 3),
 ((8298, 54653), 3),
 ((8315, 52656), 3),
 ((10696, 76630), 3),
 ((10697, 76

In [29]:
sim_tuples

[((18065, 123203), 7),
 ((18065, 123204), 7),
 ((18065, 123205), 7),
 ((18065, 123206), 7),
 ((18065, 124132), 7),
 ((18065, 124133), 7),
 ((18065, 124134), 7),
 ((18065, 124135), 7),
 ((18065, 124136), 7),
 ((18065, 124137), 7),
 ((18065, 124138), 7),
 ((19783, 135289), 7),
 ((19783, 135290), 7),
 ((6157, 43527), 6),
 ((6157, 43528), 6),
 ((6157, 43529), 6),
 ((6157, 43530), 6),
 ((6157, 43531), 6),
 ((6157, 43532), 6),
 ((6157, 43533), 6),
 ((6157, 43534), 6),
 ((6157, 43535), 6),
 ((6157, 46262), 6),
 ((6157, 46263), 6),
 ((6157, 46264), 6),
 ((6157, 46265), 6),
 ((6157, 46266), 6),
 ((6157, 46267), 6),
 ((6157, 46268), 6),
 ((6157, 46308), 6),
 ((8251, 53357), 6),
 ((8251, 57309), 6),
 ((8251, 57310), 6),
 ((8255, 57309), 6),
 ((8255, 57310), 6),
 ((8256, 57309), 6),
 ((8256, 57310), 6),
 ((18063, 123203), 6),
 ((18063, 123204), 6),
 ((18063, 123205), 6),
 ((18063, 123206), 6),
 ((18063, 124132), 6),
 ((18063, 124133), 6),
 ((18063, 124134), 6),
 ((18063, 124135), 6),
 ((18063, 124

In [30]:
pd.DataFrame(sim_tuples)

,0,1
0,"(18065, 123203)",7
1,"(18065, 123204)",7
2,"(18065, 123205)",7
3,"(18065, 123206)",7
4,"(18065, 124132)",7
...,...,...
46136863,"(54728, 372896)",1
46136864,"(54728, 372897)",1
46136865,"(54728, 372898)",1
46136866,"(54728, 372899)",1


How do Gor'kii's *Mother* and the Synodal Bible overlap? 
windows of 3-5 words with 1-2+ of overlap